# [How_to_count_tokens_with_tiktoken.ipynb](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb)

In [50]:
import openai
from dotenv import dotenv_values

openai.api_key = dotenv_values('../.env')['OPENAI_API_KEY']

In [3]:
import tiktoken

In [17]:
encoding1 = tiktoken.get_encoding("p50k_base")
encoding2 = tiktoken.get_encoding("cl100k_base")

In [5]:
encoding.encode("tiktoken is great!")

[83, 1609, 5963, 374, 2294, 0]

In [6]:
len(encoding.encode("tiktoken is great!"))

6

In [7]:
book_text = """
All children, except one, grow up. They soon know that they will grow
up, and the way Wendy knew was this. One day when she was two years old
she was playing in a garden, and she plucked another flower and ran
with it to her mother. I suppose she must have looked rather
delightful, for Mrs. Darling put her hand to her heart and cried, “Oh,
why can’t you remain like this for ever!” This was all that passed
between them on the subject, but henceforth Wendy knew that she must
grow up. You always know after you are two. Two is the beginning of the
end.

Of course they lived at 14, and until Wendy came her mother was the
chief one. She was a lovely lady, with a romantic mind and such a sweet
mocking mouth. Her romantic mind was like the tiny boxes, one within
the other, that come from the puzzling East, however many you discover
there is always one more; and her sweet mocking mouth had one kiss on
it that Wendy could never get, though there it was, perfectly
conspicuous in the right-hand corner.

"""

In [18]:
len(encoding1.encode(book_text))

252

In [19]:
len(encoding2.encode(book_text))

245

***

In [15]:
encoding1 = tiktoken.encoding_for_model('text-davinci-003')
encoding2 = tiktoken.encoding_for_model('gpt-3.5-turbo')

In [16]:
len(encoding1.encode(book_text))

252

In [20]:
len(encoding2.encode(book_text))

245

***

In [23]:
def num_tokens_from_model(text:str, model_name:str) -> int:
    encoding = tiktoken.encoding_for_model(model_name=model_name)
    num_token = len(encoding.encode(text))
    
    return num_token

In [24]:
num_tokens_from_model(book_text, 'text-davinci-003')

252

In [25]:
num_tokens_from_model(book_text, 'gpt-3.5-turbo')

245

In [28]:
num_tokens_from_model(book_text, 'gpt-3.5-turbo-16k')

245

In [29]:
num_tokens_from_model(book_text, 'gpt-4')

245

In [30]:
num_tokens_from_model(book_text, 'gpt-4-32k')

245

***

# Count tokens for ChatCompletion API

In [39]:
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613"):
    """Return the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model in {
        "gpt-3.5-turbo-0613",
        "gpt-3.5-turbo-16k-0613",
        "gpt-4-0314",
        "gpt-4-32k-0314",
        "gpt-4-0613",
        "gpt-4-32k-0613",
        }:
        tokens_per_message = 3
        tokens_per_name = 1
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif "gpt-3.5-turbo" in model:
#         print("Warning: gpt-3.5-turbo may update over time. Returning num tokens assuming gpt-3.5-turbo-0613.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613")
    elif "gpt-4" in model:
#         print("Warning: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.")
        return num_tokens_from_messages(messages, model="gpt-4-0613")
    else:
        raise NotImplementedError(
            f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens."""
        )
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens

In [40]:
example_messages = [
    {
        "role": "system",
        "content": "You are a helpful, pattern-following assistant that translates corporate jargon into plain English.",
    },
    {
        "role": "system",
        "name": "example_user",
        "content": "New synergies will help drive top-line growth.",
    },
    {
        "role": "system",
        "name": "example_assistant",
        "content": "Things working well together will increase revenue.",
    },
    {
        "role": "system",
        "name": "example_user",
        "content": "Let's circle back when we have more bandwidth to touch base on opportunities for increased leverage.",
    },
    {
        "role": "system",
        "name": "example_assistant",
        "content": "Let's talk later when we're less busy about how to do better.",
    },
    {
        "role": "user",
        "content": "This late pivot means we don't have time to boil the ocean for the client deliverable.",
    },
]

In [41]:
num_tokens_from_messages(example_messages, 'gpt-3.5-turbo')

129

***

In [44]:
from tabulate import tabulate

In [47]:
data = []
headers = ["Model Name", "Num Tokens"]

color_code = "\033[1;31m"
bold_code = "\033[1m"

for model in [
    "gpt-3.5-turbo-0301",
    "gpt-3.5-turbo-0613",
    "gpt-3.5-turbo",
    "gpt-4-0314",
    "gpt-4-0613",
    "gpt-4",]:
    
    num_tokens = num_tokens_from_messages(example_messages, model)
    colored_and_bold_model_name = f"{color_code}{bold_code}{model}\033[0m"
    data.append([colored_and_bold_model_name, num_tokens])

table = tabulate(data, headers, tablefmt="grid")
print(table)

+--------------------+--------------+
| Model Name         |   Num Tokens |
+====================+==============+
| gpt-3.5-turbo-0301 |          127 |
+--------------------+--------------+
| gpt-3.5-turbo-0613 |          129 |
+--------------------+--------------+
| gpt-3.5-turbo      |          129 |
+--------------------+--------------+
| gpt-4-0314         |          129 |
+--------------------+--------------+
| gpt-4-0613         |          129 |
+--------------------+--------------+
| gpt-4              |          129 |
+--------------------+--------------+


In [51]:
data = []
headers = ["Model Name", "Num Tokens (num_tokens_from_messages())", "Num Tokens (OpenAI API)"]

for model in [
    "gpt-3.5-turbo-0301",
    "gpt-3.5-turbo-0613",
    "gpt-3.5-turbo",
]:
    num_tokens = num_tokens_from_messages(example_messages, model)
    response = openai.ChatCompletion.create(
        model=model,
        messages=example_messages,
        temperature=0,
        max_tokens=1,
    )
    api_num_tokens = response["usage"]["prompt_tokens"]
    data.append([model, num_tokens, api_num_tokens])

table = tabulate(data, headers, tablefmt="grid")
print(table)


+--------------------+-------------------------------------------+---------------------------+
| Model Name         |   Num Tokens (num_tokens_from_messages()) |   Num Tokens (OpenAI API) |
+====================+===========================================+===========================+
| gpt-3.5-turbo-0301 |                                       127 |                       127 |
+--------------------+-------------------------------------------+---------------------------+
| gpt-3.5-turbo-0613 |                                       129 |                       129 |
+--------------------+-------------------------------------------+---------------------------+
| gpt-3.5-turbo      |                                       129 |                       127 |
+--------------------+-------------------------------------------+---------------------------+
